In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

from datetime import date, datetime, timedelta
import time

t0 = time.time()

# Read recipe inputs
NAFCUSTOMER_REVENUE_AGGREGATED = dataiku.Dataset("NAFCUSTOMER_REVENUE_AGGREGATED")
NAFCUSTOMER_REVENUE_AGGREGATED_df = NAFCUSTOMER_REVENUE_AGGREGATED.get_dataframe()

t1 = time.time()
print("load duration", (t1-t0)/60.0, "minutes")

print(len(NAFCUSTOMER_REVENUE_AGGREGATED_df))
NAFCUSTOMER_REVENUE_AGGREGATED_df.head()

49718631


,CUSTOMER_ID,CUSTOMER,REVENUE_DATE,SETUP_DATE,BI_CUS_PAR,BI_PRODUCT,BI_MRU,REVENUE_AMOUNT_USD,GROSS_SPEND_AMOUNT,PURCHASE_GALLONS_QTY,TOTAL_ACTIVE_CARD_COUNT
0,77473242.0,PROMINENCE ELECTRIC LLC,2021-09-01 00:00:00.000,2021-01-26 08:00:00+00:00,21000.0,15100.0,4000.0,5.860000,1055.08,413.960,0.0
1,77614536.0,USDA FLEET SP3,2022-08-29 00:00:00.000,2018-10-08 07:00:00+00:00,51000.0,15308.0,3100.0,-13893.139089,NaN,NaN,0.0
2,77272410.0,HARWOOD BAPTIST CHURCH,2020-08-01 00:00:00.000,2019-12-13 08:00:00+00:00,11300.0,15308.0,4120.0,0.000000,0.00,0.000,0.0
3,77133439.0,CITY OF BOERNE,2023-01-01 00:00:00.000,2021-06-04 07:00:00+00:00,13000.0,15102.0,4000.0,157.480000,6023.50,1935.509,25.0
4,77208672.0,ENCON CONSTRUCTION LLC,2020-10-15 00:00:00.000,2018-09-13 07:00:00+00:00,12200.0,15102.0,4120.0,-0.471743,NaN,NaN,0.0


In [3]:
print(len(NAFCUSTOMER_REVENUE_AGGREGATED_df))
df = NAFCUSTOMER_REVENUE_AGGREGATED_df

49718631


In [4]:
df.head()

,CUSTOMER_ID,CUSTOMER,REVENUE_DATE,SETUP_DATE,BI_CUS_PAR,BI_PRODUCT,BI_MRU,REVENUE_AMOUNT_USD,GROSS_SPEND_AMOUNT,PURCHASE_GALLONS_QTY,TOTAL_ACTIVE_CARD_COUNT
0,77473242.0,PROMINENCE ELECTRIC LLC,2021-09-01 00:00:00.000,2021-01-26 08:00:00+00:00,21000.0,15100.0,4000.0,5.860000,1055.08,413.960,0.0
1,77614536.0,USDA FLEET SP3,2022-08-29 00:00:00.000,2018-10-08 07:00:00+00:00,51000.0,15308.0,3100.0,-13893.139089,NaN,NaN,0.0
2,77272410.0,HARWOOD BAPTIST CHURCH,2020-08-01 00:00:00.000,2019-12-13 08:00:00+00:00,11300.0,15308.0,4120.0,0.000000,0.00,0.000,0.0
3,77133439.0,CITY OF BOERNE,2023-01-01 00:00:00.000,2021-06-04 07:00:00+00:00,13000.0,15102.0,4000.0,157.480000,6023.50,1935.509,25.0
4,77208672.0,ENCON CONSTRUCTION LLC,2020-10-15 00:00:00.000,2018-09-13 07:00:00+00:00,12200.0,15102.0,4120.0,-0.471743,NaN,NaN,0.0


In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

#TENURE_CALC_df = NAFCUSTOMER_REVENUE_AGGREGATED_df # For this sample code, simply copy input to output


# Write recipe outputs
#TENURE_CALC = dataiku.Dataset("TENURE_CALC")
#TENURE_CALC.write_with_schema(TENURE_CALC_df)